# 1. 엑셀데이터 불러오기

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import numpy as np
import re
import os
from pyvirtualdisplay import Display
from selenium import webdriver


df_input = pd.read_csv('../input_data_~0407 (version 1).csv')
df_input.iloc[0]

pl_id                                          1
pl_maker                                   apple
pl_group                                   A2221
pl_model_code                              아이폰11
pl_name                                 iphone11
pl_model_name                                NaN
나무위키링크단품         https://namu.wiki/w/iPhone%2011
Unnamed: 7                                   NaN
Unnamed: 8                                   NaN
Name: 0, dtype: object

# 2. 리뷰크롤링

In [8]:
import time
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                      #스펙테이블 아이디
    "pl_model_code": "",              #모델코드
    "pl_name": "",                    #모델한글
    
    "star": "",                       #별점
    "market": "",                     #구입처(ex 인터파크, 11번가)
    "write_id": "",                   #작성자(일부가림처리)
    "upload_day": "",                 #업로드날짜
    "title":"",                       #리뷰제목 
    "text": "" ,                       #리뷰내용 
    "URL":""                          #리뷰 URL
}

crawl_data = pd.DataFrame(my_dict,index=[0])
crawl_data_none = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

def search_by_category(df_input,crawl_data,index,option):
    if option == "skt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000246&frm=NVSHMDL&iq='
    elif option == "kt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000247&frm=NVSHMDL&iq='
    elif option == "lg":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000248&frm=NVSHMDL&iq='
    else:
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000204&frm=NVSHMDL&iq='
    url2 = '&origQuery&pagingIndex=1&pagingSize=40&productSet=model&query&sort=rel&timestamp=&viewType=list'
    driver.get(url1+df_input.iloc[index][4]+url2)
    driver.implicitly_wait(3)
    
    
    #페이지에 있는 링크 다 긁어오기, 일단 스크롤기능 추가안해서 5개만 가져옴 => 5개 넘어가면 검색정확도 떨어져서 고정
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')
    div_list = soup.select(".basicList_info_area__17Xyo")
    
    link_list = []
    for i in range(len(div_list)):
        if div_list[i].select("div.basicList_info_area__17Xyo > div.basicList_etc_box__1Jzg6 > a") == []:
            pass
        else:
            link_list.append(div_list[i].select(".basicList_link__1MaTN")[0])
    

    for link in link_list:
        url = 'window.open("' + link.get('href') +'");'
        driver.execute_script(url)
        driver.switch_to.window(driver.window_handles[-1])
        req = driver.page_source
        soup=BeautifulSoup(req, 'html.parser')
        review_list = soup.select(".reviewItems_list_review__1sgcJ")
        if len(review_list)>0:
            review_list = review_list[0].find_all("li")
        
        for review in review_list:
            area1 = review.select(".reviewItems_etc_area__2P8i3")
            area1 = area1[0].find_all("span")
            
            star = area1[0].text.replace("평점","") 
            market = area1[3].text 
            write_id = area1[4].text 
            upload_day = area1[5].text 
            
            area2 = review.select(".reviewItems_review_text__2Bwpa")
            title = area2[0].select(".reviewItems_title__39Z8H")[0].text
            text = area2[0].select(".reviewItems_text__XIsTc")[0].text
            
            my_dict = {
                "pl_id": df_input.iloc[index]['pl_id'],              #기종아이디(자체부여한것)
                "pl_model_code": df_input.iloc[index]['pl_model_code'],      #모델코드
                "pl_name": df_input.iloc[index]['pl_name'],            #모델영문명

                "star": star,                       #별점
                "market": market,                   #구입처(ex 인터파크, 11번가)
                "write_id": write_id,               #작성자(일부가림처리)
                "upload_day": upload_day,           #업로드날짜
                "title":title,                      #리뷰제목 
                "text": text,                       #리뷰내용 
                "URL": driver.current_url
            }
            
            if my_dict is None or my_dict == None:
                pass
            else:
                a = pd.DataFrame(my_dict,index=[0])
                crawl_data = crawl_data.append(a)
                
                
        
        # 현재 탭 닫기
        driver.close()
        # 맨 처음 탭으로 변경(0번 탭)
        driver.switch_to.window(driver.window_handles[0])
    #display(crawl_data)
    return crawl_data

# 초기화 --------------------------------------------
display = Display(visible=0, size=(800, 600))
display.start()
driver = webdriver.Chrome()
driver.implicitly_wait(3)

for i in range(181,len(df_input)):
    if i % 30 == 0 and i != 0:
        
        time.sleep(60)
        crawl_data.to_csv('review_data_'+str(i)+'.csv',encoding='utf-8-sig')
    print(i)
    crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"all"))
    


crawl_data = crawl_data[crawl_data['pl_id'] != '']
crawl_data = crawl_data.reset_index(drop=True)
crawl_data.to_csv('review_data_0409'+'.csv',encoding='utf-8-sig')
    
    
    

181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


In [4]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 5000)
pd.set_option('display.max_columns', 100)
crawl_data

pandas version:  1.1.5


,pl_id,pl_model_code,pl_name,star,market,write_id,upload_day,title,text,URL
0,1,아이폰11,NaN,5,하이마트쇼핑몰,z*o*****,21.02.13.,토요일 배송 될거라 생각했는데,토요일 배송 될거라 생각했는데 월요일 배송되었네요.아이 졸업 선물로 카메라 좋은 거...,https://search.shopping.naver.com/catalog/2085...
0,1,아이폰11,NaN,4,스마트베이비,ciao****,21.01.10.,"실버없어서 골드로 주문했는데, 기존 가지고있는거랑 비슷하지만 맘에 들어요~ 통신사 ...","실버없어서 골드로 주문했는데, 기존 가지고있는거랑 비슷하지만 맘에 들어요~ 통신사 ...",https://search.shopping.naver.com/catalog/2085...
0,1,아이폰11,NaN,4,11번가,lo******,20.08.09.,좋아요,"배송빠르고 좋았어요.단, 아이폰이 스크래치있는 초기불량제품이어서 뽑기를 잘못한듯.매...",https://search.shopping.naver.com/catalog/2085...
0,1,아이폰11,NaN,5,G마켓,k*0*****,20.11.21.,완전 적극적으로 추천합니다,골드로 주문하였는데 품절이라 미드나잇으로 보내주셨습니다~ 추가금 없이 업그레이드 받...,https://search.shopping.naver.com/catalog/2085...
0,1,아이폰11,NaN,5,애플 공식 브랜드스토어,1004****,20.10.04.,"구매 후 2틀만에 도착했습니다. 출고가와 많이 차이나진 않지만, 일반 휴대폰 매장에...","구매 후 2틀만에 도착했습니다. 출고가와 많이 차이나진 않지만, 일반 휴대폰 매장에...",https://search.shopping.naver.com/catalog/2085...
0,1,아이폰11,NaN,5,프렌드가젯,flow****,20.11.29.,여러군데 알아보고 어렵게 샀는데요 궁금한 점도 빠른 답변으로 응대해주시고 상품도 정...,여러군데 알아보고 어렵게 샀는데요 궁금한 점도 빠른 답변으로 응대해주시고 상품도 정...,https://search.shopping.naver.com/catalog/2085...
0,1,아이폰11,NaN,3,하이마트쇼핑몰,k*n*****,20.07.08.,아.. 진짜 딥빡이네요\n스마트픽 하,아.. 진짜 딥빡이네요스마트픽 하지마세요.개인간 택배거래해도 물건 뽁뽁이에 싸주는게...,https://search.shopping.naver.com/catalog/2085...
0,1,아이폰11,NaN,5,SSG닷컴,lo******,20.02.07.,테두리: 지문 묻는 실버 뒷면:,테두리: 지문 묻는 실버 뒷면: 지문 묻지 않는 화이트 크기: 손에 착 감기는 맛용...,https://search.shopping.naver.com/catalog/2085...
0,1,아이폰11,NaN,4,위메프,pik*****,20.07.28.,아이폰 12살까하다가,아이폰 12살까하다가11pro 구매했습니다 ㅎㅎ 이쁩니다알뜰폰으로 같이 사용하려고 ...,https://search.shopping.naver.com/catalog/2085...
0,1,아이폰11,NaN,5,SSG닷컴,lj******,19.12.27.,카드 할인으로 15만원 할인 받고,카드 할인으로 15만원 할인 받고 제가 원하는 색상과 용량을 구매해서 너무 만족합니...,https://search.shopping.naver.com/catalog/2085...


In [53]:
crawl_data.to_csv('review_data1.csv.csv',encoding='utf-8-sig')

# 엑셀에서 pl_id 오름차순, text_len(추가후) 내림차순 정렬하고 난 후 => file.csv

In [108]:
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                    #모델영문명
    "text1": "",
    "text2": "",
    "text3": ""
}

filtered_reivew = pd.DataFrame(my_dict,index=[0])
filtered_reivew_none = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

df_output = pd.read_csv('file.csv')
pl_ids = df_output['pl_id'].unique()
models = []
for pl_id in pl_ids:
    is_name = df_output['pl_id'] == pl_id
    model = df_output[is_name]
    my_dict = {
    "pl_id": pl_id,                 
    }
    
    print(len(model))
    if len(model) > 0:
        my_dict["text1"] = model.iloc[0]['text']
    if len(model) > 1:
        my_dict["text2"] = model.iloc[1]['text']
    if len(model) > 2:
        my_dict["text3"] = model.iloc[2]['text']
    
    filtered_reivew = filtered_reivew.append(pd.DataFrame(my_dict,index=[0]))

89
67
60
80
44
72
77
76
43
86
22
62
70
10
68
8
20
21
20
42
17
15
20
3
25
22
29
45
14
26
44
100
55
14
57
100
10
8
8
2
4
8
23
1
77
80
88
81
89
74
80
39
81
54
42
82
100
85
13
13
36
33
100
85
37
32
25
7
71
12
1
56
16
92
12
6
20
45
86
32
20
20
10
10
1
10
7
11
4
13
10
27
1
8
8
8
4
1
2
63
35
45
21
42
31
16
3
20
20
20
3
3
5
1
1
10
10
5
4
48
6
52
65
55
28
23
20
20
21
21
20
49
29
20
22
1
35
35
35
35
4
13
100
20
20
15
16
12
3
20


In [110]:
filtered_reivew.to_csv('filtered_reivew.csv',encoding='utf-8-sig')